## Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.
## Mobile Games AB Testing with Cookie Cats

In [1]:
from scipy import stats
import numpy as np

import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [3]:
df = pd.read_csv('результаты А_B - cookie_cats 2.csv')
df

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,0,0
1,337,gate_30,38,1,0
2,377,gate_40,165,1,0
3,483,gate_40,1,0,0
4,488,gate_40,179,1,1
...,...,...,...,...,...
90184,9999441,gate_40,97,1,0
90185,9999479,gate_40,30,0,0
90186,9999710,gate_30,28,1,0
90187,9999768,gate_40,51,1,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  int64 
 4   retention_7     90189 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 3.4+ MB


In [5]:
df.describe()

,userid,sum_gamerounds,retention_1,retention_7
count,9.018900e+04,90189.000000,90189.000000,90189.000000
mean,4.998412e+06,51.872457,0.445210,0.186065
std,2.883286e+06,195.050858,0.496992,0.389161
min,1.160000e+02,0.000000,0.000000,0.000000
25%,2.512230e+06,5.000000,0.000000,0.000000
50%,4.995815e+06,16.000000,0.000000,0.000000
75%,7.496452e+06,51.000000,1.000000,0.000000
max,9.999861e+06,49854.000000,1.000000,1.000000


Набор данных не содержит никаких нулевых данных.

In [6]:
df['userid'].count()

90189

In [7]:
df['userid'].nunique()

90189

Дубликатов ID в наборе данных нет.

In [9]:
df.version.value_counts()

gate_40    45489
gate_30    44700
Name: version, dtype: int64

Число версий примерно одинаковое. Сравним версии gate_30 и gate_40, визульно они очень похожи.

In [10]:
df[df['version']=='gate_30'].describe()

,userid,sum_gamerounds,retention_1,retention_7
count,4.470000e+04,44700.000000,44700.000000,44700.000000
mean,4.987564e+06,52.456264,0.448188,0.190201
std,2.881026e+06,256.716423,0.497314,0.392464
min,1.160000e+02,0.000000,0.000000,0.000000
25%,2.505469e+06,5.000000,0.000000,0.000000
50%,4.983631e+06,17.000000,0.000000,0.000000
75%,7.481497e+06,50.000000,1.000000,0.000000
max,9.999710e+06,49854.000000,1.000000,1.000000


In [11]:
df[df['version']=='gate_40'].describe()

,userid,sum_gamerounds,retention_1,retention_7
count,4.548900e+04,45489.000000,45489.000000,45489.000000
mean,5.009073e+06,51.298776,0.442283,0.182000
std,2.885496e+06,103.294416,0.496663,0.385849
min,3.770000e+02,0.000000,0.000000,0.000000
25%,2.517171e+06,5.000000,0.000000,0.000000
50%,5.007329e+06,16.000000,0.000000,0.000000
75%,7.510762e+06,52.000000,1.000000,0.000000
max,9.999861e+06,2640.000000,1.000000,1.000000


In [22]:
from scipy.stats import ttest_1samp, mannwhitneyu, shapiro, kstest, ttest_ind

In [19]:
# Проверим нормальность выборок.

print(kstest(df['sum_gamerounds'], 'norm'))

KstestResult(statistic=0.8715607041848303, pvalue=0.0, statistic_location=2, statistic_sign=-1)


p-value имеет очень низкое значение, значит, распределение не нормальное.

In [20]:

mannwhitneyu(df[df.version == 'gate_30'].sum_gamerounds,
             df[df.version == 'gate_40'].sum_gamerounds)

MannwhitneyuResult(statistic=1024331250.5, pvalue=0.05020880772044255)

Гипотезы:
H0: Набор статистически значимо не влияет (p-value > 0.05)
H1: Набор статистически значимо влияет (p-value < 0.05)
p-value выше значения альфы 0,05, значит, мы не отвергаем нулевую гипотезу, что статистически значимых различий нет. Однако, значение p-value очень близко к альфе, используем другие критерии.

In [21]:
ttest_ind(df[df.version == 'gate_30'].sum_gamerounds,
          df[df.version == 'gate_40'].sum_gamerounds)

Ttest_indResult(statistic=0.8910426211362967, pvalue=0.37290868247405207)

p-value значительно выше значения альфы, что подтверждает, статистически значимых различий нет.

In [26]:
k1 = df[df['version']=='gate_30']['retention_1'].sum()
k2 = df[df['version']=='gate_40']['retention_1'].sum()

In [27]:
n1 = df[df['version']=='gate_30'].shape[0]
n2 = df[df['version']=='gate_40'].shape[0]

In [30]:
from statsmodels.stats import proportion

z_score, z_pvalue = proportion.proportions_ztest(np.array([k1, k2]), 
                                                   np.array([n1, n2]))
print('Results are ','z_score =%.3f, pvalue = %.3f'%(z_score, z_pvalue))

Results are  z_score =1.784, pvalue = 0.074


In [31]:
if abs(z_pvalue) < 0.05:
    print("We may reject the null hypothesis!")
else:
    print("We have failed to reject the null hypothesis")

We have failed to reject the null hypothesis


Непараметрический Хи-квадрат

In [32]:
chisq, pvalue, table = proportion.proportions_chisquare(np.array([k1, k2]), 
                                                   np.array([n1, n2]))

print('Results are ','chisq =%.3f, pvalue = %.3f'%(chisq, pvalue))

Results are  chisq =3.183, pvalue = 0.074


In [33]:
if abs(pvalue) < 0.05:
    print("We may reject the null hypothesis!")
else:
    print("We have failed to reject the null hypothesis")

We have failed to reject the null hypothesis


Итак, все рассмотренные критерии позволяют сделать вывод, что статистически значимых различий нет, p-value превышает значение альфы.

In [34]:
# Определим количество пользователей, кто играл 1 день и далее продолжал играть
df[(df['retention_1'] == 1) & (df['retention_7'] == 1)]

,userid,version,sum_gamerounds,retention_1,retention_7
4,488,gate_40,179,1,1
5,540,gate_40,187,1,1
8,1574,gate_40,108,1,1
14,2218,gate_30,305,1,1
17,2451,gate_30,204,1,1
...,...,...,...,...,...
90125,9991408,gate_40,186,1,1
90127,9991540,gate_30,117,1,1
90134,9991949,gate_30,191,1,1
90150,9995412,gate_40,253,1,1


In [35]:
# Определим количество пользователей, кто играл 1 день и далее не продолжал играть
df[(df['retention_1'] == 1) & (df['retention_7'] == 0)]

,userid,version,sum_gamerounds,retention_1,retention_7
1,337,gate_30,38,1,0
2,377,gate_40,165,1,0
9,1587,gate_40,153,1,0
12,2132,gate_40,30,1,0
13,2179,gate_30,39,1,0
...,...,...,...,...,...
90181,9998733,gate_40,10,1,0
90182,9999178,gate_30,21,1,0
90184,9999441,gate_40,97,1,0
90186,9999710,gate_30,28,1,0


In [36]:
# Определим количество пользователей, кто играл в течение недели, но не в первый день
df[(df['retention_1'] == 0) & (df['retention_7'] == 1)]

,userid,version,sum_gamerounds,retention_1,retention_7
10,1842,gate_40,3,0,1
23,2729,gate_30,105,0,1
90,8788,gate_30,19,0,1
102,10200,gate_40,106,0,1
143,13429,gate_30,60,0,1
...,...,...,...,...,...
89967,9975174,gate_30,26,0,1
90044,9982773,gate_30,1,0,1
90046,9983040,gate_30,137,0,1
90156,9996093,gate_40,23,0,1


In [37]:
# Определим количество пользователей, кто вообще не играл
df[(df['retention_1'] == 0) & (df['retention_7'] == 0)]

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,0,0
3,483,gate_40,1,0,0
6,1066,gate_30,0,0,0
7,1444,gate_40,2,0,0
11,2101,gate_30,0,0,0
...,...,...,...,...,...
90178,9998555,gate_40,32,0,0
90180,9998623,gate_30,7,0,0
90183,9999349,gate_30,10,0,0
90185,9999479,gate_40,30,0,0


26970 пользователей (29,9%) играли только в 1-й день, 13181 пользователей (14,62%) играли в 1-й  день и далее продолжали играть, 3598 пользователей (3,99%) играли в течение недели, но не в первый день. И 46436 пользователей вообще не играли, а это 51,5%. Однако, раунды у не игравших есть, возможно стоит пересмотреть алгоритмы подсчета, поскольку более 50% таких результатов. Среди не игравших в 1-й день, но игравших в течение недели, есть количество раундов по значению меньше, чем у вообще не игравших. Таким образом, статистически значимых различий нет, но не совсем понятны алгоритмы подсчета, возможно просто стоит провести модернизацию контрольной версии.